https://mp.weixin.qq.com/s/9tgZSrE5rc3vedbDlcHETQ  
### 视频处理入门  
从相机中读取视频
通常情况下，我们必须用摄像机捕捉实时画面。提供了一个非常简单的界面。让我们从摄像头捕捉一段视频(我使用的是我笔记本电脑内置的网络摄像头) ，将其转换成灰度视频并显示出来。只是一个简单的任务开始。  
要捕获视频，你需要创建一个 VideoCapture 对象。它的参数可以是设备索引或视频文件的名称。设备索引就是指定哪个摄像头的数字。正常情况下，一个摄像头会被连接(就像我的情况一样)。所以我简单地传0(或-1)。你可以通过传递1来选择第二个相机，以此类推。在此之后，你可以逐帧捕获。但是在最后，不要忘记释放俘虏。

In [8]:
import numpy as np
import cv2 as cv

In [10]:
cap = cv.VideoCapture(0)
#检查cap有无初始化捕获
if not cap.isOpened():
    print("Cannot open camera")
    exit()
while True:
    # 逐帧捕获
    ret, frame = cap.read()
    # ret 读取了数据就返回True,没有读取数据(已到尾部)就返回False
    # frame 返回读取的视频数据 ——  一帧数据是一个三维数组
    if not ret:
        print("Can't receive frame (stream end?). Exiting ...")
        break
    # 我们在框架上的操作到这里
    #cv.cvtColor将图像灰度化
    gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)#注释掉会变成灰度图片，不会是视频输出
    # 显示结果帧e
    cv.imshow('frame', gray)
    if cv.waitKey(1) == ord('q'):
        break
# 完成所有操作后，释放捕获器
cap.release()
cv.destroyAllWindows()

cap.read()返回布尔值(True/ False)。如果正确读取了帧，它将为True。因此，你可以通过检查此返回值来检查视频的结尾。  
有时，cap可能尚未初始化捕获。在这种情况下，此代码显示错误。你可以通过cap.isOpened()方法检查它是否已初始化。如果是True，那么确定。否则，使用cap.open()打开它。  
你还可以使用cap.get(propId)方法访问该视频的某些功能，其中propId是0到18之间的一个数字。每个数字表示视频的属性（如果适用于该视频），并且可以显示完整的详细信息在这里看到：cv::Video，Capture::get()。其中一些值可以使用cap.set(propId，value)进行修改。value是你想要的新值。  
例如，我可以通过cap.get(cv.CAP_PROP_FRAME_WIDTH)和cap.get(cv.CAP_PROP_FRAME_HEIGHT)检查框架的宽度和高度。默认情况下，它的分辨率为640x480。但我想将其修改为320x240。只需使用和即可。  
cap.get(cv.CAP_PROP_FRAME_WIDTH)  
cap.get(cv.CAP_PROP_FRAME_HEIGHT)    
ret = cap.set(cv.CAP_PROP_FRAME_WIDTH,320)  
ret = cap.set(cv.CAP_PROP_FRAME_HEIGHT,240).

### 从文件播放视频  
它与从相机捕获相同，只是用视频文件名更改摄像机索引。另外，在显示框架时，请使用适当的时间cv.waitKey()。如果太小，则视频将非常快，而如果太大，则视频将变得很慢（嗯，这就是显示慢动作的方式）。正常情况下25毫秒就可以了。

In [6]:
cap = cv.VideoCapture('2022GDJSXB325金蝉脱壳.mp4')
while cap.isOpened():
    ret, frame = cap.read()
    # 如果正确读取帧，ret为True
    if not ret:
        print("Can't receive frame (stream end?). Exiting ...")
        break
    gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
    cv.imshow('frame', gray)
    if cv.waitKey(1) == ord('q'):
        break
cap.release()
cv.destroyAllWindows()

### 保存视频  
所以我们捕捉一个视频，一帧一帧地处理，我们想要保存这个视频。对于图像，它非常简单，只需使用 cv.imwrite()。这里还需要做一些工作。  
这次我们创建一个 VideoWriter 对象。我们应该指定输出文件名(例如: output.avi)。然后我们应该指定 FourCC 代码(详见下一段)。然后传递帧率的数量和帧大小。最后一个是颜色标志。如果为 True，编码器期望颜色帧，否则它与灰度帧一起工作。  
FourCC：http://en.wikipedia.org/wiki/FourCC 是用于指定视频编解码器的4字节代码。可用代码列表可在fourcc.org中:http://www.fourcc.org/codecs.php 找到。它取决于平台。遵循编解码器对我来说效果很好。  
在Fedora中：DIVX，XVID，MJPG，X264，WMV1，WMV2。（最好使用XVID。MJPG会生成大尺寸的视频。X264会生成非常小的尺寸的视频）
在Windows中：DIVX（尚待测试和添加）  
在OSX中：MJPG（.mp4），DIVX（.avi），X264（.mkv）。  
FourCC代码作为MJPG的cv.VideoWriter_fourcc（'M'，'J'，'P'，'G'）or cv.VideoWriter_fourcc（*'MJPG'）传递。  
在从摄像机捕获的代码下面，沿垂直方向翻转每一帧并保存。  

In [4]:
cap = cv.VideoCapture(0)
# 定义编解码器并创建VideoWriter对象
fourcc = cv.VideoWriter_fourcc(*'XVID')
out = cv.VideoWriter('output.avi', fourcc, 20.0, (640,  480))
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("Can't receive frame (stream end?). Exiting ...")
        break
    # 写翻转的框架
    frame = cv.flip(frame, 0)
    '''
    Img = cv.flip(img,1,dst=None) #水平镜像
    Img = cv.flip(img,0,dst=None) #垂直镜像
    Img = cv.flip(img,-1,dst=None) #对角镜像
    '''
    out.write(frame)
    cv.imshow('frame', frame)
    if cv.waitKey(1) == ord('q'):
        break
# 完成工作后释放所有内容
cap.release()
out.release()
cv.destroyAllWindows()